In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt # 득점모델 변수 중요도

import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
%matplotlib inline

In [42]:
pd.set_option('display.max_columns', None)

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [29]:
##train데이터 불러오기
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [30]:
train_df["label"] = np.nan

In [31]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y_01 Feature..
test_x = test_df.filter(regex='X')
test_y = test_df.filter(regex='Y')

In [32]:
# 범위로 나누기
# conditionlist = [
#     (train_df['X_49'] <10000) ,
#     (train_df['X_49'] >=10000) & (train_df['X_49'] <50000),
#     (train_df['X_49'] >= 50000)]
# choicelist = [1, 2, 3]
# train_df["label"] = np.select(conditionlist, choicelist, default='Not Specified')

In [45]:
train_df.drop([ 'X_48', 'X_47', 'X_23', 'X_11', 'X_10', 'X_04'], inplace= True, axis=1)
test_df.drop([ 'X_48', 'X_47', 'X_23', 'X_11', 'X_10', 'X_04'], inplace= True, axis=1)

In [46]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components = 5) # Gaussian Mixture 객체 생성
gmm.fit(train_x)

GaussianMixture(n_components=5)

In [51]:
gmm_cluster_labels_tr = gmm.predict(train_x)
gmm_cluster_labels_te = gmm.predict(test_x)

In [60]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, random_state=0) # K-means
kmeans.fit(train_x)

KMeans(n_clusters=5, random_state=0)

In [61]:
kmm_cluster_labels_tr = kmeans.predict(train_x)
kmm_cluster_labels_te = kmeans.predict(test_x)

In [56]:
train_df["label"] = gmm_cluster_labels_tr
test_df["label"] = gmm_cluster_labels_te

In [62]:
train_df["km_label"] = kmm_cluster_labels_tr
test_df["km_label"] = kmm_cluster_labels_te

In [71]:
col_Y = train_df.columns[-15:-1]
train_df.groupby([ "km_label"])[col_Y].mean()

,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14,label
km_label,,,,,,,,,,,,,,
0,1.067044,1.030071,13.429678,31.199658,16.543063,3.158171,-26.363663,-26.378223,-22.395614,24.238424,-26.307330,-26.301497,-26.312872,2.127652
1,1.075716,1.026069,13.634023,31.326869,16.578802,3.152149,-26.273549,-26.289801,-22.317927,24.322901,-26.218604,-26.214391,-26.226577,1.241805
2,1.095103,1.080906,13.597373,31.315753,16.780330,3.173923,-26.234755,-26.241043,-22.221560,24.449560,-26.178665,-26.166221,-26.184979,1.591760
3,1.117878,1.095322,14.794872,32.466700,16.791894,3.049467,-26.156417,-26.168089,-22.120156,24.520956,-26.075389,-26.090322,-26.104089,1.000000
4,1.033785,0.995433,13.624887,31.251460,16.462729,3.158275,-26.310937,-26.322154,-22.499063,24.334762,-26.251632,-26.248454,-26.260118,1.905997


In [58]:
col_X = train_df.columns[1:-16]
train_df.groupby("label")[col_X].mean()

,X_01,X_02,X_03,X_05,X_06,X_07,X_08,X_09,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_49,X_50,X_51,X_52,X_53,X_54,X_55
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,69.008866,103.320156,71.326984,102.428318,70.905646,28.690420,135.505111,231.722676,4.378555,0.141069,13.366784,13.380546,13.453645,13.506219,13.441544,3.296063,3.240895,3.228137,3.292262,2.110997,2.087616,2.081429,2.087342,2.105539,2.157211,1.372453,1.535627,1.355920,1.554385,12.950435,12.919431,12.941884,12.919337,-15.800779,-15.821178,-16.453265,21.184498,21.057160,21.191473,21.158848,0.147788,1468.364425,15921.669029,130.595057,131.874326,138.545296,128.316438,127.845824,137.438886
1,69.102988,103.320164,68.824818,102.335497,71.503852,31.564279,273.928092,198.317311,4.369090,0.149592,13.366573,13.382478,13.455274,13.514835,13.448060,3.251637,3.205900,3.188182,3.253080,2.117806,2.091974,2.103082,2.107327,2.141367,2.196313,1.402881,1.599825,1.385193,1.641530,12.952219,12.920711,12.942495,12.917757,-15.855829,-15.858428,-16.507295,21.179314,21.040721,21.198084,21.146086,0.164589,1468.160227,21117.034411,130.904471,131.822725,138.352394,128.371320,128.229135,136.714962
2,68.834337,103.320134,70.768551,102.382282,70.949670,30.743768,704.376312,251.955459,4.378213,0.144058,13.370966,13.376538,13.467729,13.503897,13.443285,3.269501,3.202206,3.195008,3.264010,2.124042,2.100531,2.095040,2.101723,2.118100,2.171385,1.374783,1.542158,1.358293,1.556312,12.946715,12.917182,12.938647,12.920676,-14.949308,-13.771192,-16.523301,21.182061,21.067778,21.180290,21.160354,0.149372,1468.864734,15873.931127,130.403561,131.442344,138.595664,127.775884,127.709075,137.680950
3,68.734551,103.320155,69.620147,102.284408,71.148915,29.162933,155.833372,223.343666,4.373519,0.144340,13.366716,13.377918,13.454311,13.509267,13.444428,3.245455,3.190440,3.181437,3.234076,2.130059,2.109677,2.097243,2.105718,2.121232,2.175777,1.367214,1.574809,1.345572,1.461554,12.951320,12.921173,12.943607,12.915308,-15.856012,-15.866393,-16.510880,21.181437,21.053842,21.196716,21.153695,0.153138,1468.316716,14586.313900,124.312410,125.965195,132.775641,122.176022,121.963055,130.452601
4,67.246757,103.320183,65.296674,102.212572,69.723764,29.331347,128.792903,228.173563,4.367627,0.143469,13.382425,13.383868,13.481893,13.520740,13.460780,3.157052,3.096399,3.092964,3.140174,2.120342,2.102682,2.096332,2.108611,2.125830,2.185885,1.377328,1.607635,1.362815,1.636148,12.949276,12.921543,12.941252,12.919534,-16.111559,-16.094692,-16.768787,21.194337,21.070716,21.224298,21.168497,0.159457,1468.178972,15735.010381,131.138262,130.855042,138.899771,127.528123,128.309616,139.233753


In [59]:
col_X = test_df.columns[1:-16]
test_df.groupby("label")[col_X].mean()

,X_01,X_02,X_03,X_05,X_06,X_07,X_08,X_09,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,68.994845,103.320155,71.364045,102.418875,70.885658,28.677799,134.608712,231.849045,4.378979,0.141212,13.366551,13.380272,13.453398,13.506174,13.441254,3.293640,3.240074,3.225582,3.291306,2.110953,2.087649,2.081339,2.087519,2.105456,2.156902,1.372490,1.535736,1.356029,1.554683,12.950574,12.919804,12.941868,12.918953,-15.796222,-15.816527
1,69.086454,103.320162,68.798124,102.354665,71.490169,31.995451,281.471101,198.960431,4.369141,0.149953,13.366361,13.382605,13.455836,13.515114,13.448361,3.250485,3.204286,3.186573,3.252068,2.118055,2.091960,2.102942,2.108397,2.141266,2.195947,1.401636,1.598397,1.384485,1.639517,12.951308,12.920024,12.942440,12.920457,-15.867977,-15.871146
2,69.197781,103.320147,71.182072,102.448787,71.217614,31.153722,758.814406,250.641751,4.376117,0.144205,13.375694,13.380644,13.474386,13.508028,13.449296,3.266942,3.204688,3.191610,3.265855,2.125151,2.100362,2.096197,2.101167,2.119014,2.170604,1.373722,1.545412,1.360101,1.554628,12.949235,12.917264,12.940624,12.922414,-15.338249,-13.959718
3,68.550256,103.320177,69.346338,102.248008,71.116259,29.210648,175.992873,228.445915,4.372225,0.144789,13.371070,13.382423,13.463296,13.513521,13.449803,3.247070,3.191155,3.181324,3.237718,2.139746,2.105408,2.098394,2.104676,2.124761,2.178254,1.364197,1.582310,1.342197,1.574901,12.951211,12.926394,12.939746,12.917465,-15.930732,-15.939775
4,67.232361,103.320177,65.295455,102.215955,69.729092,29.354905,127.942634,226.857719,4.367724,0.143384,13.382523,13.383845,13.482548,13.520848,13.461126,3.156173,3.096373,3.091985,3.140586,2.120199,2.102696,2.096396,2.108677,2.126019,2.186192,1.377419,1.607956,1.362739,1.636446,12.949034,12.921524,12.941096,12.919751,-16.107726,-16.089445


In [55]:
train_df.groupby("label")[col_X].mean()

,X_01,X_02,X_03,X_05,X_06,X_07,X_08,X_09,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_49,X_50,X_51,X_52,X_53,X_54,X_55
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,69.008866,103.320156,71.326984,102.428318,70.905646,28.690420,135.505111,231.722676,4.378555,0.141069,13.366784,13.380546,13.453645,13.506219,13.441544,3.296063,3.240895,3.228137,3.292262,2.110997,2.087616,2.081429,2.087342,2.105539,2.157211,1.372453,1.535627,1.355920,1.554385,12.950435,12.919431,12.941884,12.919337,-15.800779,-15.821178,-16.453265,21.184498,21.057160,21.191473,21.158848,0.147788,1468.364425,15921.669029,130.595057,131.874326,138.545296,128.316438,127.845824,137.438886
1,69.102988,103.320164,68.824818,102.335497,71.503852,31.564279,273.928092,198.317311,4.369090,0.149592,13.366573,13.382478,13.455274,13.514835,13.448060,3.251637,3.205900,3.188182,3.253080,2.117806,2.091974,2.103082,2.107327,2.141367,2.196313,1.402881,1.599825,1.385193,1.641530,12.952219,12.920711,12.942495,12.917757,-15.855829,-15.858428,-16.507295,21.179314,21.040721,21.198084,21.146086,0.164589,1468.160227,21117.034411,130.904471,131.822725,138.352394,128.371320,128.229135,136.714962
2,68.834337,103.320134,70.768551,102.382282,70.949670,30.743768,704.376312,251.955459,4.378213,0.144058,13.370966,13.376538,13.467729,13.503897,13.443285,3.269501,3.202206,3.195008,3.264010,2.124042,2.100531,2.095040,2.101723,2.118100,2.171385,1.374783,1.542158,1.358293,1.556312,12.946715,12.917182,12.938647,12.920676,-14.949308,-13.771192,-16.523301,21.182061,21.067778,21.180290,21.160354,0.149372,1468.864734,15873.931127,130.403561,131.442344,138.595664,127.775884,127.709075,137.680950
3,68.734551,103.320155,69.620147,102.284408,71.148915,29.162933,155.833372,223.343666,4.373519,0.144340,13.366716,13.377918,13.454311,13.509267,13.444428,3.245455,3.190440,3.181437,3.234076,2.130059,2.109677,2.097243,2.105718,2.121232,2.175777,1.367214,1.574809,1.345572,1.461554,12.951320,12.921173,12.943607,12.915308,-15.856012,-15.866393,-16.510880,21.181437,21.053842,21.196716,21.153695,0.153138,1468.316716,14586.313900,124.312410,125.965195,132.775641,122.176022,121.963055,130.452601
4,67.246757,103.320183,65.296674,102.212572,69.723764,29.331347,128.792903,228.173563,4.367627,0.143469,13.382425,13.383868,13.481893,13.520740,13.460780,3.157052,3.096399,3.092964,3.140174,2.120342,2.102682,2.096332,2.108611,2.125830,2.185885,1.377328,1.607635,1.362815,1.636148,12.949276,12.921543,12.941252,12.919534,-16.111559,-16.094692,-16.768787,21.194337,21.070716,21.224298,21.168497,0.159457,1468.178972,15735.010381,131.138262,130.855042,138.899771,127.528123,128.309616,139.233753


In [ ]:
t